In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from sklearn import datasets, metrics
# from sklearn.preprocessing import LabelEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import accuracy_score
# import joblib
# from scipy.signal import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from scipy.stats import kurtosis, skew

# import sqlite3
import math 
# import json
# import datetime
# import calendar
# import os

# import pylab
# import matplotlib.colors as colors
# import matplotlib.cm as cm
# import itertools as it

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; } .CodeMirror pre {font-size: 9pt;}</style>"))

In [ ]:
recs = 1
rounds = 12
trial_len = 20000

rec_start_times = [1656969540000]
rec_end_times = [1656970020000]

# rec_start_times = [1656664800000,1656665400000,1656665820000,1656667020000,1656667320000,1656667920000,1656668400000,1656668880000,1656669300000,1656672240000,1656672720000,1656673020000,1656673560000,1656673860000,1656674520000,1656674820000]
# rec_end_times = [1656665100000,1656665700000,1656666120000,1656667320000,1656667620000,1656668220000,1656668700000,1656669180000,1656669600000,1656672540000,1656673020000,1656673320000,1656673860000,1656674160000,1656674820000,1656675120000]

In [ ]:
math_df = pd.DataFrame()
rest_df = pd.DataFrame()

trial_num = 0
trial_rows = math.inf

for i in range(recs):
  df = pd.read_csv('./'+str(rec_start_times[i])+'.'+str(rec_end_times[i])+'.csv')
  df.drop(df[(df.timestamp < rec_start_times[i]) | (df.timestamp > rec_end_times[i])].index, inplace=True)

  for j in range(rounds):
    round_start = rec_start_times[i] + j*trial_len*2

    # split round into math trial and rest trial
    math_trial = df[(df.timestamp > round_start) & (df.timestamp < round_start + trial_len)]
    rest_trial = df[(df.timestamp > round_start + trial_len) & (df.timestamp < round_start + 2*trial_len)]

    # number rows of trial
    math_trial['row'] = range(len(math_trial))
    rest_trial['row'] = range(len(rest_trial))

    # find min rows as trials are jagged
    if len(math_trial) < trial_rows:
      trial_rows = len(math_trial)
    if len(rest_trial) < trial_rows:
      trial_rows = len(rest_trial)

    # label overall trial number
    math_trial['trial'] = trial_num
    rest_trial['trial'] = trial_num
    trial_num += 1

    # display(math_trial)
    # add trials to respective dataframes
    math_df = math_df.append(math_trial)
    rest_df = rest_df.append(rest_trial)
  print(trial_rows)

  # display(math_df)

492


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
from scipy.stats import linregress

windows = 11
win_rows = 2*int(trial_rows/(windows+1))
# print(trial_rows)
# print(win_rows)

labels = []
cols = np.array([])
for w in range(windows):
  cols = np.append(cols, ['hbo_slope_'+str(w), 'hbo_max_'+str(w), 'hbo_min_'+str(w), 'hbo_mean_'+str(w), 'hbo_std_'+str(w), 'hbo_skew_'+str(w), 'hbo_kurt_'+str(w)])
  cols = np.append(cols, ['hbr_slope_'+str(w), 'hbr_max_'+str(w), 'hbr_min_'+str(w), 'hbr_mean_'+str(w), 'hbr_std_'+str(w), 'hbr_skew_'+str(w), 'hbr_kurt_'+str(w)])
  # cols = np.append(cols, ['hbo/r_corr'+str(w)])

feat_df = pd.DataFrame(columns=cols)

for t in range(trial_num):
  math_feat = []
  rest_feat = []

  # labels
  labels.append(1)
  labels.append(0)

  for w in range(windows):
    win_start = w*win_rows/2
    win_end = win_start + win_rows
 
    math_win = math_df[(math_df.trial == t) & (math_df.row >= win_start) & (math_df.row < win_end)]
    # print('trial: ', t)
    # print('win: ', w)
    
    # hbo features
    math_feat.append(linregress(math_win.timestamp, math_win.hbo)[0])
    math_feat.append(math_win.hbo.max())
    math_feat.append(math_win.hbo.min())
    math_feat.append(math_win.hbo.mean())
    math_feat.append(math_win.hbo.std())
    math_feat.append(skew(math_win.hbo))
    math_feat.append(kurtosis(math_win.hbo))

    # hbr features
    math_feat.append(linregress(math_win.timestamp, math_win.hbr)[0])
    math_feat.append(math_win.hbr.max())
    math_feat.append(math_win.hbr.min())
    math_feat.append(math_win.hbr.mean())
    math_feat.append(math_win.hbr.std())
    math_feat.append(skew(math_win.hbr))
    math_feat.append(kurtosis(math_win.hbr))
    
    # math_feat.append(math_win.hbo.corr(math_win.hbr))


    rest_win = rest_df[(rest_df.trial == t) & (rest_df.row >= win_start) & (rest_df.row < win_end)]

    # hbr features
    rest_feat.append(linregress(rest_win.timestamp, rest_win.hbo)[0])
    rest_feat.append(rest_win.hbo.max())
    rest_feat.append(rest_win.hbo.min())
    rest_feat.append(rest_win.hbo.mean())
    rest_feat.append(rest_win.hbo.std())
    rest_feat.append(skew(rest_win.hbo))
    rest_feat.append(kurtosis(rest_win.hbo))

    # hbr features
    rest_feat.append(linregress(rest_win.timestamp, rest_win.hbr)[0])
    rest_feat.append(rest_win.hbr.max())
    rest_feat.append(rest_win.hbr.min())
    rest_feat.append(rest_win.hbr.mean())
    rest_feat.append(rest_win.hbr.std())
    rest_feat.append(skew(rest_win.hbr))
    rest_feat.append(kurtosis(rest_win.hbr))

    # rest_feat.append(rest_win.hbo.corr(rest_win.hbr))


  feat_df.loc[2*t] = math_feat
  feat_df.loc[2*t+1] = rest_feat

display(feat_df)

,hbo_slope_0,hbo_max_0,hbo_min_0,hbo_mean_0,hbo_std_0,hbo_skew_0,hbo_kurt_0,hbr_slope_0,hbr_max_0,hbr_min_0,...,hbo_std_10,hbo_skew_10,hbo_kurt_10,hbr_slope_10,hbr_max_10,hbr_min_10,hbr_mean_10,hbr_std_10,hbr_skew_10,hbr_kurt_10
0,-1.992359e-18,7.958452e-13,7.641891e-13,7.786548e-13,7.101839e-15,-0.067535,-0.689022,-1.992359e-18,7.958452e-13,7.641891e-13,...,6.676910e-15,0.738084,1.131648,-3.624905e-18,7.667458e-13,7.280784e-13,7.419994e-13,6.676910e-15,0.738084,1.131648
1,-1.168937e-19,7.548016e-13,7.308478e-13,7.428958e-13,5.517383e-15,-0.092074,-0.545755,-1.168937e-19,7.548016e-13,7.308478e-13,...,6.774343e-15,0.377917,0.075362,2.922114e-18,7.725773e-13,7.369445e-13,7.515346e-13,6.774343e-15,0.377917,0.075362
2,-2.119804e-19,7.631405e-13,7.354029e-13,7.518146e-13,6.076362e-15,-0.585474,-0.024962,-2.119804e-19,7.631405e-13,7.354029e-13,...,4.949891e-15,-0.363017,-0.472416,-1.241052e-18,7.505858e-13,7.289727e-13,7.410585e-13,4.949891e-15,-0.363017,-0.472416
3,3.244920e-18,7.505794e-13,7.254114e-13,7.385714e-13,5.849656e-15,-0.180806,-0.481781,3.244920e-18,7.505794e-13,7.254114e-13,...,5.606448e-15,-0.135629,-0.807647,1.249675e-18,7.605942e-13,7.379453e-13,7.505769e-13,5.606448e-15,-0.135629,-0.807647
4,1.585062e-18,7.671060e-13,7.444469e-13,7.552198e-13,4.957231e-15,0.135485,-0.466543,1.585062e-18,7.671060e-13,7.444469e-13,...,4.243852e-15,-0.208062,-0.317395,-7.651224e-19,7.486880e-13,7.289530e-13,7.403256e-13,4.243852e-15,-0.208062,-0.317395
5,1.606665e-18,7.496841e-13,7.286382e-13,7.402883e-13,4.130276e-15,-0.182048,0.046950,1.606665e-18,7.496841e-13,7.286382e-13,...,3.688009e-15,0.103814,-0.228926,2.351327e-18,7.335175e-13,7.169898e-13,7.255141e-13,3.688009e-15,0.103814,-0.228926
6,-4.686907e-19,7.375876e-13,7.204629e-13,7.307768e-13,3.233869e-15,-0.657730,0.480162,-4.686907e-19,7.375876e-13,7.204629e-13,...,3.327886e-15,0.470194,0.073193,1.803041e-18,7.387615e-13,7.216243e-13,7.288804e-13,3.327886e-15,0.470194,0.073193
7,1.144699e-18,7.402815e-13,7.252272e-13,7.330151e-13,3.601464e-15,-0.211867,-0.640667,1.144699e-18,7.402815e-13,7.252272e-13,...,2.712570e-15,-0.127023,-0.475612,-6.602187e-19,7.294805e-13,7.175666e-13,7.244030e-13,2.712570e-15,-0.127023,-0.475612
8,9.650363e-19,7.296193e-13,7.159225e-13,7.240291e-13,2.936278e-15,-0.663757,0.008548,9.650363e-19,7.296193e-13,7.159225e-13,...,3.241956e-15,-0.085328,-0.669096,-2.206330e-18,7.346164e-13,7.218618e-13,7.288458e-13,3.241956e-15,-0.085328,-0.669096
9,1.142629e-18,7.426912e-13,7.230628e-13,7.333248e-13,3.970119e-15,-0.187579,0.115821,1.142629e-18,7.426912e-13,7.230628e-13,...,3.273283e-15,-0.397671,-0.359019,-8.883333e-19,7.392069e-13,7.240239e-13,7.321014e-13,3.273283e-15,-0.397671,-0.359019


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feat_df, labels, test_size=0.2)

classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)

from sklearn.metrics import classification_report, confusion_matrix

y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[ 7  5]
 [10 10]]
              precision    recall  f1-score   support

           0       0.41      0.58      0.48        12
           1       0.67      0.50      0.57        20

    accuracy                           0.53        32
   macro avg       0.54      0.54      0.53        32
weighted avg       0.57      0.53      0.54        32



In [ ]:
|hbo1 = FNIRSDATA3['hbo'].to_numpy()
hbo2 = FNIRSDATA4['hbo'].to_numpy()

dd_hbo = np.array([hbo1[0:1000], hbo1[1500:2500], hbo1[3000:4000], hbo1[4500:5500], hbo1[6000:7000], hbo1[7500:8500], hbo1[9000:10000], hbo1[10500:11500], hbo1[12000:13000], hbo1[13500:14500], 
          hbo2[0:1000], hbo2[1500:2500], hbo2[3000:4000], hbo2[4500:5500], hbo2[6000:7000], hbo2[7500:8500], hbo2[9000:10000], hbo2[10500:11500], hbo2[12000:13000], hbo2[13500:14500]])

rest_hbo_500 = np.array([hbo1[1000:1500], hbo1[2500:3000], hbo1[4000:4500], hbo1[5500:6000], hbo1[7000:7500], hbo1[8500:9000], hbo1[10000:10500], hbo1[11500:12000], hbo1[13000:13500], hbo1[14500:15000],
            hbo2[1000:1500], hbo2[2500:3000], hbo2[4000:4500], hbo2[5500:6000], hbo2[7000:7500], hbo2[8500:9000], hbo2[10000:10500], hbo2[11500:12000], hbo2[13000:13500], hbo2[14500:15000]] )

rest_hbo = [np.repeat(rest_hbo_500[0], 2), np.repeat(rest_hbo_500[1], 2), np.repeat(rest_hbo_500[2], 2), np.repeat(rest_hbo_500[3], 2), np.repeat(rest_hbo_500[4], 2), np.repeat(rest_hbo_500[5], 2), np.repeat(rest_hbo_500[6], 2), np.repeat(rest_hbo_500[7], 2), np.repeat(rest_hbo_500[8], 2), np.repeat(rest_hbo_500[9], 2),
            np.repeat(rest_hbo_500[10], 2), np.repeat(rest_hbo_500[11], 2), np.repeat(rest_hbo_500[12], 2), np.repeat(rest_hbo_500[13], 2), np.repeat(rest_hbo_500[14], 2), np.repeat(rest_hbo_500[15], 2), np.repeat(rest_hbo_500[16], 2), np.repeat(rest_hbo_500[17], 2), np.repeat(rest_hbo_500[18], 2), np.repeat(rest_hbo_500[19], 2)]

hbo_win = np.empty((40,9,200))

for i in range(20):
  hbo_win[i] = [dd_hbo[i][0:200], dd_hbo[i][100:300], dd_hbo[i][200:400], dd_hbo[i][300:500], dd_hbo[i][400:600], dd_hbo[i][500:700], dd_hbo[i][600:800], dd_hbo[i][700:900], dd_hbo[i][800:1000]]
  hbo_win[i+20] = [rest_hbo[i][0:200], rest_hbo[i][100:300], rest_hbo[i][200:400], rest_hbo[i][300:500], rest_hbo[i][400:600], rest_hbo[i][500:700], rest_hbo[i][600:800], rest_hbo[i][700:900], rest_hbo[i][800:1000]]

# dd_hbo[trial(0-19)][window(0-8)][value(0-199)]

SyntaxError: ignored

In [ ]:
from scipy.stats import linregress

hbo_win_max = np.empty((40,9))
hbo_win_mean = np.empty((40,9))
hbo_win_std = np.empty((40,9))
hbo_win_skew = np.empty((40,9))
hbo_win_kurt = np.empty((40,9))

linregress(a, b)
for i in range(40):
  for j in range(9):
    # print(hbo_win[i,j].shape)
    hbo_win_max[i,j] = np.max(hbo_win[i,j])
    hbo_win_mean[i,j] = np.mean(hbo_win[i,j])
    hbo_win_std[i,j] = np.std(hbo_win[i,j])
    hbo_win_skew[i,j] = skew(hbo_win[i,j])
    hbo_win_kurt[i,j] = kurtosis(hbo_win[i,j])

hbo = np.concatenate((hbo_win_max, hbo_win_mean, hbo_win_std, hbo_win_skew, hbo_win_kurt), axis=1)
# print(hbo.shape)

col = np.empty((45), str)
for i in range(9):
  col[i] = 'hbo max ' + str(i+1)
  col[i+9] = 'hbo mean ' + str(i+1)
  col[i+18] = 'hbo std ' + str(i+1)
  col[i+27] = 'hbo skew ' + str(i+1)
  col[i+36] = 'hbo kurt ' + str(i+1)
# print(len(col))
df = pd.DataFrame(hbo, columns=col)
labels = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

# display(df)